## 사용하는 패키지들

In [43]:
import json
import pandas as pd

import re

## 데이터 로드

In [25]:
with open('../Datasets/train.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [26]:
train_data = pd.DataFrame(json_data)
train_data = train_data.drop(['id', 'plylst_title', 'updt_date', 'like_cnt'], axis=1)
train_data.head()

,tags,songs
0,[락],"[525514, 129701, 383374, 562083, 297861, 13954..."
1,"[추억, 회상]","[432406, 675945, 497066, 120377, 389529, 24427..."
2,"[까페, 잔잔한]","[83116, 276692, 166267, 186301, 354465, 256598..."
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...","[394031, 195524, 540149, 287984, 440773, 10033..."
4,[댄스],"[159327, 553610, 5130, 645103, 294435, 100657,..."


In [27]:
with open('../Datasets/song_meta.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [28]:
song_data = pd.DataFrame(json_data)
song_data = song_data.drop(['album_name', 'song_gn_gnr_basket'], axis=1)
song_data.head()

,song_gn_dtl_gnr_basket,issue_date,album_id,artist_id_basket,song_name,artist_name_basket,id
0,[GN0901],20140512,2255639,[2727],Feelings,[Various Artists],0
1,"[GN1601, GN1606]",20080421,376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[Murray Perahia],1
2,[GN0901],20180518,4698747,[3361],Solsbury Hill (Remastered 2002),[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[Matoma],3
4,"[GN1802, GN1801]",20110824,2008470,[560160],그남자 그여자,[Jude Law],4


## 데이터 열 이름 변경

In [29]:
train_data.rename(columns={'songs':'song_id'}, inplace=True)
train_data.head()

,tags,song_id
0,[락],"[525514, 129701, 383374, 562083, 297861, 13954..."
1,"[추억, 회상]","[432406, 675945, 497066, 120377, 389529, 24427..."
2,"[까페, 잔잔한]","[83116, 276692, 166267, 186301, 354465, 256598..."
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...","[394031, 195524, 540149, 287984, 440773, 10033..."
4,[댄스],"[159327, 553610, 5130, 645103, 294435, 100657,..."


In [30]:
song_data.rename(columns={'id':'song_id', 'song_gn_dtl_gnr_basket': 'gnr'}, inplace=True)
song_data = song_data.astype({'issue_date':'int64'})
song_data.head()

,gnr,issue_date,album_id,artist_id_basket,song_name,artist_name_basket,song_id
0,[GN0901],20140512,2255639,[2727],Feelings,[Various Artists],0
1,"[GN1601, GN1606]",20080421,376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[Murray Perahia],1
2,[GN0901],20180518,4698747,[3361],Solsbury Hill (Remastered 2002),[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[Matoma],3
4,"[GN1802, GN1801]",20110824,2008470,[560160],그남자 그여자,[Jude Law],4


## 데이터 추출

- 500개의 플레이리스트 추출

In [31]:
train_data_sample = train_data[:500]

## 데이터 병합

In [32]:
train_data_explode = train_data_sample.explode('song_id', ignore_index=True)
train_data_explode.head()

,tags,song_id
0,[락],525514
1,[락],129701
2,[락],383374
3,[락],562083
4,[락],297861


In [33]:
train_dict = dict()

for i in range(len(train_data_explode)):
    song = train_data_explode['song_id'][i]
    tag = train_data_explode['tags'][i]
    
    if song in train_dict:
        for j in tag:
            train_dict[song].add(j)
    
    else:
        train_dict[song] = set(tag)
        
print(train_dict[157435])

{'스트레스해소', '걸그룹댄스', '댄스', 'kpop', '여자아이돌'}


In [34]:
train_data_explode.drop_duplicates(subset='song_id', keep='first',inplace=True)
train_data_explode.shape

(16674, 2)

In [35]:
for i in range(len(train_data_explode)):
    song = train_data_explode['song_id'].iloc[i]
    
    train_data_explode['tags'].iloc[i] = list(train_dict[song])

train_data_explode.head()

,tags,song_id
0,[락],525514
1,[락],129701
2,[락],383374
3,[락],562083
4,[락],297861


In [36]:
song_tag_appended = pd.merge(train_data_explode, song_data)
song_tag_appended = song_tag_appended.astype({'song_id':'int64'})
song_tag_appended.head()

,tags,song_id,gnr,issue_date,album_id,artist_id_basket,song_name,artist_name_basket
0,[락],525514,"[GN1402, GN1401]",20130506,2200223,[734201],Hey Little Girl,[The Sol]
1,[락],129701,"[GN0901, GN0902, GN1001]",20130917,2201802,[536907],Octagon,[Royal Bangs]
2,[락],383374,"[GN1012, GN1005, GN1001]",19911021,2216938,[166978],The Road,[Honeymoon Suite]
3,[락],562083,"[GN1013, GN0901, GN0902, GN1001]",20000919,43227,[19035],Honeymoon,[Phoenix]
4,[락],297861,"[GN1013, GN0901, GN0902, GN1001]",20050306,303657,[170117],High,[James Blunt]


In [37]:
song_tag_appended.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16674 entries, 0 to 16673
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tags                16674 non-null  object
 1   song_id             16674 non-null  int64 
 2   gnr                 16674 non-null  object
 3   issue_date          16674 non-null  int64 
 4   album_id            16674 non-null  int64 
 5   artist_id_basket    16674 non-null  object
 6   song_name           16674 non-null  object
 7   artist_name_basket  16674 non-null  object
dtypes: int64(3), object(5)
memory usage: 1.1+ MB


## 데이터 전처리

## 태그 세트 생성

In [60]:
def make_tag_set(data):
    tag_set = set()

    for i in range(len(data)):
        tag = data['tags'][i]
    
        for j in tag:
            tag_set.add(j)

    return tag_set

In [61]:
tag_set = make_tag_set(train_data_sample)
print(tag_set)

{'클래식', '프로필', '조용하게', '대만', '여유', '음악추천', '따듯한', 'Pop', 'BlackMusic', '맥주', '달달하게', '연주음악', '연휴', '기분전환', '외로운', '퇴근길', '락', '아련하게', '슬픔의', '생기', '성탄절', '새해', '상쾌한', '야경', '떼껄룩', '벚꽃', 'Billboard', '정서발달', '쇼핑', '록발라드', '존윌리엄스', '태교음악', '일상', 'electronica', '모닝콜', '아이돌명곡', 'OST', '추천', '눈', '집중', '월드', '7월', '뷰티샵', '연주곡', '국힙', 'Jazz', '릴랙스요가클래식', '호호클럽', '후회', '인디뮤직', '심포닉메탈', '차막힘', '수록곡', '스포츠', '뉴에이지', 'HIPSTER', '동요', '주간일렉트로니카', '겨울노래', '록', '날려버려', '셧업앤댄스', '커피', '신진래퍼', '고백', '펑키', '휴가', '가을여행', '띵곡들', '커플', '이별', '국내힙합', '분위기', 'jazzymood', '여름방학', '사이다', '몽글', '신나는곡', '달달한', '2000', '시험기간집중할때', '나만알고싶은', '기쁠때', '북카페', '부모님', '키즈동화', '댄스뮤직', '섹시', '중국', '메탈', '추억의', '혼자만의겨울', '헬스장에서운동할때듣는노래', '백색소음', '트로피컬하우스', '라운지', '우울한', '비', '쓸쓸', '칠뮤직', '스티비원더', '자장가', '유럽여행', '그냥좋은노래', '여자가수', '9살', '8', '잔잔한', '듣고', '공부하며듣는', '고민', '부부뮤지션', '드림팝', '시간순', '나만알고싶은음악', 'EDMFloor', '노래추천', '국외', 'Chill', '가을감성', '겨울왕국', '나만들을꺼', '쓸쓸함', '아미를위한', '해외알앤비소울', '콘서트', '아티스트', '타이틀곡', '레깅스', '고음

## 태그 dict 생성 함수

## 영어 대소문자 통일

- 소문자로 통일

In [41]:
def make_dict_lower(tag_set):
    tag_dict = dict()
    
    for i in tag_set:
        if not i in tag_dict:
            tag_dict[i] = i.lower()
    
    return tag_dict

In [42]:
tag_lower_dict = make_dict_lower(tag_set)
print(tag_lower_dict)

{'클래식': '클래식', '프로필': '프로필', '조용하게': '조용하게', '대만': '대만', '여유': '여유', '음악추천': '음악추천', '따듯한': '따듯한', 'Pop': 'pop', 'BlackMusic': 'blackmusic', '맥주': '맥주', '달달하게': '달달하게', '연주음악': '연주음악', '연휴': '연휴', '기분전환': '기분전환', '외로운': '외로운', '퇴근길': '퇴근길', '락': '락', '아련하게': '아련하게', '슬픔의': '슬픔의', '생기': '생기', '성탄절': '성탄절', '새해': '새해', '상쾌한': '상쾌한', '야경': '야경', '떼껄룩': '떼껄룩', '벚꽃': '벚꽃', 'Billboard': 'billboard', '정서발달': '정서발달', '쇼핑': '쇼핑', '록발라드': '록발라드', '존윌리엄스': '존윌리엄스', '태교음악': '태교음악', '일상': '일상', 'electronica': 'electronica', '모닝콜': '모닝콜', '아이돌명곡': '아이돌명곡', 'OST': 'ost', '추천': '추천', '눈': '눈', '집중': '집중', '월드': '월드', '7월': '7월', '뷰티샵': '뷰티샵', '연주곡': '연주곡', '국힙': '국힙', 'Jazz': 'jazz', '릴랙스요가클래식': '릴랙스요가클래식', '호호클럽': '호호클럽', '후회': '후회', '인디뮤직': '인디뮤직', '심포닉메탈': '심포닉메탈', '차막힘': '차막힘', '수록곡': '수록곡', '스포츠': '스포츠', '뉴에이지': '뉴에이지', 'HIPSTER': 'hipster', '동요': '동요', '주간일렉트로니카': '주간일렉트로니카', '겨울노래': '겨울노래', '록': '록', '날려버려': '날려버려', '셧업앤댄스': '셧업앤댄스', '커피': '커피', '신진래퍼': '신진래퍼', '고백': '고백', '펑키': '펑키', '휴가': '휴가

## 특수문자 제거

In [44]:
def make_dict_special(tag_set):
    tag_dict = dict()
    
    for i in tag_set:
        if not i in tag_dict:
            # 아래를 제외한 문자는 ''로 대체한다
            # \uAC00-WuD7A3 : 모든 한글 음절
            # 0-9 : 숫자
            # a-zA-Z : 모든 영어
            tag_dict[i] = re.sub(r'[^\uAC00-\uD7A30-9a-zA-Z]', '', i)
    
    return tag_dict

In [46]:
tag_special_dict = make_dict_special(tag_set)

# 현재 태그들은 특수문자가 '_'만 존재한다
print(tag_special_dict)

{'클래식': '클래식', '프로필': '프로필', '조용하게': '조용하게', '대만': '대만', '여유': '여유', '음악추천': '음악추천', '따듯한': '따듯한', 'Pop': 'Pop', 'BlackMusic': 'BlackMusic', '맥주': '맥주', '달달하게': '달달하게', '연주음악': '연주음악', '연휴': '연휴', '기분전환': '기분전환', '외로운': '외로운', '퇴근길': '퇴근길', '락': '락', '아련하게': '아련하게', '슬픔의': '슬픔의', '생기': '생기', '성탄절': '성탄절', '새해': '새해', '상쾌한': '상쾌한', '야경': '야경', '떼껄룩': '떼껄룩', '벚꽃': '벚꽃', 'Billboard': 'Billboard', '정서발달': '정서발달', '쇼핑': '쇼핑', '록발라드': '록발라드', '존윌리엄스': '존윌리엄스', '태교음악': '태교음악', '일상': '일상', 'electronica': 'electronica', '모닝콜': '모닝콜', '아이돌명곡': '아이돌명곡', 'OST': 'OST', '추천': '추천', '눈': '눈', '집중': '집중', '월드': '월드', '7월': '7월', '뷰티샵': '뷰티샵', '연주곡': '연주곡', '국힙': '국힙', 'Jazz': 'Jazz', '릴랙스요가클래식': '릴랙스요가클래식', '호호클럽': '호호클럽', '후회': '후회', '인디뮤직': '인디뮤직', '심포닉메탈': '심포닉메탈', '차막힘': '차막힘', '수록곡': '수록곡', '스포츠': '스포츠', '뉴에이지': '뉴에이지', 'HIPSTER': 'HIPSTER', '동요': '동요', '주간일렉트로니카': '주간일렉트로니카', '겨울노래': '겨울노래', '록': '록', '날려버려': '날려버려', '셧업앤댄스': '셧업앤댄스', '커피': '커피', '신진래퍼': '신진래퍼', '고백': '고백', '펑키': '펑키', '휴가': '휴가

## Stopword 제거

- 현재 태그는 단순 단어나 띄어쓰기가 제거된 단어의 조합으로 구성
- 임의의 Stopword를 선정할 필요가 존재

In [52]:
# 설정 필요
stopwords = {'날려버려'}

In [55]:
def make_dict_stop(tag_set, stopwords):
    tag_dict = dict()
    
    for i in tag_set:
        if not i in tag_dict:
            # 빈 칸으로 만든 뒤 다른 함수로 제거할 예정
            if i in stopwords:
                tag_dict[i] = ''
            else:
                tag_dict[i] = i
                
    return tag_dict

In [54]:
tag_stop_dict = make_dict_stop(tag_set, stopwords)
print(tag_stop_dict)

{'클래식': '클래식', '프로필': '프로필', '조용하게': '조용하게', '대만': '대만', '여유': '여유', '음악추천': '음악추천', '따듯한': '따듯한', 'Pop': 'Pop', 'BlackMusic': 'BlackMusic', '맥주': '맥주', '달달하게': '달달하게', '연주음악': '연주음악', '연휴': '연휴', '기분전환': '기분전환', '외로운': '외로운', '퇴근길': '퇴근길', '락': '락', '아련하게': '아련하게', '슬픔의': '슬픔의', '생기': '생기', '성탄절': '성탄절', '새해': '새해', '상쾌한': '상쾌한', '야경': '야경', '떼껄룩': '떼껄룩', '벚꽃': '벚꽃', 'Billboard': 'Billboard', '정서발달': '정서발달', '쇼핑': '쇼핑', '록발라드': '록발라드', '존윌리엄스': '존윌리엄스', '태교음악': '태교음악', '일상': '일상', 'electronica': 'electronica', '모닝콜': '모닝콜', '아이돌명곡': '아이돌명곡', 'OST': 'OST', '추천': '추천', '눈': '눈', '집중': '집중', '월드': '월드', '7월': '7월', '뷰티샵': '뷰티샵', '연주곡': '연주곡', '국힙': '국힙', 'Jazz': 'Jazz', '릴랙스요가클래식': '릴랙스요가클래식', '호호클럽': '호호클럽', '후회': '후회', '인디뮤직': '인디뮤직', '심포닉메탈': '심포닉메탈', '차막힘': '차막힘', '수록곡': '수록곡', '스포츠': '스포츠', '뉴에이지': '뉴에이지', 'HIPSTER': 'HIPSTER', '동요': '동요', '주간일렉트로니카': '주간일렉트로니카', '겨울노래': '겨울노래', '록': '록', '날려버려': '', '셧업앤댄스': '셧업앤댄스', '커피': '커피', '신진래퍼': '신진래퍼', '고백': '고백', '펑키': '펑키', '휴가': '휴가', '

## Stemming 제거

- 한글에서 동일한 의미여도 표현이 다른 경우를 자동적으로 찾는 것은 불가능하다
- 임의로 동일한 의미의 단어 묶음을 만들 필요가 있다

In [50]:
# 설정 필요
stemmings = {
    '락' : {'락', '록'}
}

In [58]:
def make_dict_stem(tag_set, stemmings):
    tag_dict = dict()
    
    for i in tag_set:
        if not i in tag_dict:
            
            # stemming으로 설정한 단어가 존재하는지 확인
            for j in stemmings:
                if i in stemmings[j]:
                    tag_dict[i] = j
                    break
            
            # 존재하지 않는 경우
            if not i in tag_dict:
                tag_dict[i] = i
    
    return tag_dict

In [59]:
tag_stem_dict = make_dict_stem(tag_set, stemmings)
print(tag_stem_dict)

{'클래식': '클래식', '프로필': '프로필', '조용하게': '조용하게', '대만': '대만', '여유': '여유', '음악추천': '음악추천', '따듯한': '따듯한', 'Pop': 'Pop', 'BlackMusic': 'BlackMusic', '맥주': '맥주', '달달하게': '달달하게', '연주음악': '연주음악', '연휴': '연휴', '기분전환': '기분전환', '외로운': '외로운', '퇴근길': '퇴근길', '락': '락', '아련하게': '아련하게', '슬픔의': '슬픔의', '생기': '생기', '성탄절': '성탄절', '새해': '새해', '상쾌한': '상쾌한', '야경': '야경', '떼껄룩': '떼껄룩', '벚꽃': '벚꽃', 'Billboard': 'Billboard', '정서발달': '정서발달', '쇼핑': '쇼핑', '록발라드': '록발라드', '존윌리엄스': '존윌리엄스', '태교음악': '태교음악', '일상': '일상', 'electronica': 'electronica', '모닝콜': '모닝콜', '아이돌명곡': '아이돌명곡', 'OST': 'OST', '추천': '추천', '눈': '눈', '집중': '집중', '월드': '월드', '7월': '7월', '뷰티샵': '뷰티샵', '연주곡': '연주곡', '국힙': '국힙', 'Jazz': 'Jazz', '릴랙스요가클래식': '릴랙스요가클래식', '호호클럽': '호호클럽', '후회': '후회', '인디뮤직': '인디뮤직', '심포닉메탈': '심포닉메탈', '차막힘': '차막힘', '수록곡': '수록곡', '스포츠': '스포츠', '뉴에이지': '뉴에이지', 'HIPSTER': 'HIPSTER', '동요': '동요', '주간일렉트로니카': '주간일렉트로니카', '겨울노래': '겨울노래', '록': '락', '날려버려': '날려버려', '셧업앤댄스': '셧업앤댄스', '커피': '커피', '신진래퍼': '신진래퍼', '고백': '고백', '펑키': '펑키', '휴가': '휴가

## 태그 dict 적용 함수

In [62]:
def apply_tag_dict(data, tag_dict):
    apply_data = data
    
    for i in range(len(apply_data)):
        tag = apply_data['tags'][i]
        apply_tag = []
        
        for j in tag:
            if not tag_dict[j] in apply_tag and tag_dict[j] != "":
                apply_tag.append(tag_dict[j])
        
        apply_data['tags'][i] = apply_tag

    return apply_data

In [63]:
lower_data = apply_tag_dict(train_data_sample, tag_lower_dict)
lower_data.head()

C:\Users\user\AppData\Local\Temp\ipykernel_3280\4061311596.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  apply_data['tags'][i] = apply_tag


,tags,song_id
0,[락],"[525514, 129701, 383374, 562083, 297861, 13954..."
1,"[추억, 회상]","[432406, 675945, 497066, 120377, 389529, 24427..."
2,"[까페, 잔잔한]","[83116, 276692, 166267, 186301, 354465, 256598..."
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...","[394031, 195524, 540149, 287984, 440773, 10033..."
4,[댄스],"[159327, 553610, 5130, 645103, 294435, 100657,..."


In [64]:
lower_tag_set = make_tag_set(lower_data)
print(lower_tag_set)

{'클래식', '프로필', '조용하게', '대만', '여유', '음악추천', '따듯한', '맥주', '달달하게', '연주음악', '연휴', '기분전환', '외로운', '퇴근길', '락', '아련하게', '슬픔의', '생기', '성탄절', '새해', '상쾌한', '야경', '떼껄룩', '벚꽃', '정서발달', '쇼핑', '록발라드', '존윌리엄스', 'asmr', '태교음악', '일상', 'electronica', 'lizzo', '모닝콜', '아이돌명곡', '추천', '눈', '집중', '월드', '7월', '뷰티샵', '연주곡', '국힙', '릴랙스요가클래식', '호호클럽', '후회', '인디뮤직', '심포닉메탈', '차막힘', '수록곡', '스포츠', '뉴에이지', '동요', '주간일렉트로니카', '겨울노래', '록', 'maroon', '날려버려', '셧업앤댄스', '신진래퍼', '고백', '커피', '휴가', '펑키', '가을여행', '띵곡들', '커플', '이별', '국내힙합', '분위기', 'jazzymood', '여름방학', '사이다', '몽글', 'feat', 'edmfloor', '신나는곡', 'elecronica', '달달한', '2000', '시험기간집중할때', '나만알고싶은', '기쁠때', '북카페', '부모님', '키즈동화', '댄스뮤직', '섹시', '중국', '메탈', '추억의', '혼자만의겨울', '헬스장에서운동할때듣는노래', '백색소음', '트로피컬하우스', '라운지', '우울한', '비', '쓸쓸', '칠뮤직', '스티비원더', '자장가', '유럽여행', '그냥좋은노래', '여자가수', 'lofi', '9살', '8', '잔잔한', '듣고', '공부하며듣는', '고민', '부부뮤지션', '드림팝', '시간순', '나만알고싶은음악', '노래추천', '국외', '가을감성', '겨울왕국', '나만들을꺼', '쓸쓸함', '아미를위한', '해외알앤비소울', '콘서트', '아티스트', '타이틀곡', '레깅스', '고음병', 'hiphop'